Code to match set of proteins from preppi database and save corresponding PPI network

In [1]:
import numpy as np
import pandas as pd
import scipy as sp
import matplotlib.pyplot as plt
import seaborn as sns
from mpl_toolkits.mplot3d import Axes3D
%matplotlib inline
import re
import time
import math
import pytest
import glob

### Just to visualise the dimension of data

In [2]:
import pandas as pd
path_to_file= "/Users/saheeba/Downloads/sorted_978.csv"
sorted_list = pd.read_csv(path_to_file, index_col=False, header=0)
sorted_list.head()

,id,Cluster ID,Unnamed: 2,Cluster name,Size,Cluster members,Organisms,Length,Identity
0,O75581,UniRef100_O75581,partially reviewed and UniParc,Cluster: Low-density lipoprotein receptor-rela...,9,O75581; B3KQA9; F5H0Z3; A0A2J8SR31; A0A2J8QQF0...,Homo sapiens (Human); Pongo abelii (Sumatran o...,1613,100%
1,P13688,UniRef100_P13688,partially reviewed and UniParc,Cluster: Carcinoembryonic antigen-related cell...,5,P13688; M0R2K4; M0R109; UPI0006C9E68E; UPI0005...,Homo sapiens (Human),526,100%
2,P00533,UniRef100_P00533,partially reviewed and UniParc,Cluster: Epidermal growth factor receptor,11,P00533; H9F140; Q75MF2; F1JTL6; C9JYS6; UPI000...,Homo sapiens (Human); Macaca mulatta (Rhesus m...,1210,100%
3,P35613,UniRef100_P35613,partially reviewed,Cluster: Basigin,6,P35613; R4GMX5; P35613-3; A0A346RDT0; A0A346RC...,Homo sapiens (Human),385,100%
4,O00592,UniRef100_O00592,reviewed,Cluster: Podocalyxin,1,O00592,Homo sapiens (Human),558,100%


In [3]:
a=(sorted_list["id"])
print(a)
sorted_list.shape

0      O75581
1      P13688
2      P00533
3      P35613
4      O00592
        ...  
973    Q14802
974    Q9UKF5
975    Q9UKQ2
976    Q86XX4
977    Q9H6B4
Name: id, Length: 978, dtype: object


(978, 9)

In [4]:
filepath= "/Users/saheeba/Downloads/preppi_final.csv"
preppi = pd.read_csv(filepath)
print(preppi.shape)
preppi.head()

(1048575, 12)


,prot1,prot2,str_score,protpep_score,str_max_score,red_score,ort_score,phy_score,coexp_score,go_score,total_score,final_score
0,Q13131,P14625,18.5900,6.44772,18.5900,4.2492,0.6153,2.4160,9.4687,10.80,12008.400,12008.400
1,P06400,Q96N96,1.8315,14.32220,14.3222,4.2492,0.0000,2.4160,2.1077,10.80,3346.930,3346.930
2,Q7Z6V5,Q8NCE0,4.5712,0.00000,4.5712,0.0000,0.0000,1.5978,9.4687,24.11,1667.400,1667.400
3,P37058,P15428,484.5640,0.00000,484.5640,1.7740,0.6153,2.4160,1.1757,2.07,3109.970,3109.970
4,O00468,Q9NR23,34.0309,0.00000,34.0309,0.5125,0.0000,2.4160,2.7714,5.86,684.323,684.323


In [5]:
from sklearn.feature_extraction.text import CountVectorizer
vect = CountVectorizer(min_df=1, lowercase=True)

X = vect.fit_transform(sorted_list["id"])
cols_emp = vect.get_feature_names()

X = vect.fit_transform(preppi['prot1'])
cols_desc = vect.get_feature_names()

common_cols_idx = [i for i,col in enumerate(cols_emp) if col in cols_desc]

preppi['Match'] = (X.toarray()[:, common_cols_idx] == 1).any(1)

print(preppi.shape)
preppi.head()

(1048575, 13)


,prot1,prot2,str_score,protpep_score,str_max_score,red_score,ort_score,phy_score,coexp_score,go_score,total_score,final_score,Match
0,Q13131,P14625,18.5900,6.44772,18.5900,4.2492,0.6153,2.4160,9.4687,10.80,12008.400,12008.400,False
1,P06400,Q96N96,1.8315,14.32220,14.3222,4.2492,0.0000,2.4160,2.1077,10.80,3346.930,3346.930,False
2,Q7Z6V5,Q8NCE0,4.5712,0.00000,4.5712,0.0000,0.0000,1.5978,9.4687,24.11,1667.400,1667.400,False
3,P37058,P15428,484.5640,0.00000,484.5640,1.7740,0.6153,2.4160,1.1757,2.07,3109.970,3109.970,False
4,O00468,Q9NR23,34.0309,0.00000,34.0309,0.5125,0.0000,2.4160,2.7714,5.86,684.323,684.323,True


In [7]:
results = preppi.query("Match==True")
export_csv = results.to_csv (r'/Users/saheeba/Downloads/aligned_result_preppi.csv', index = None, header=True) 
print(results.shape)
results.head()

(48706, 13)


,prot1,prot2,str_score,protpep_score,str_max_score,red_score,ort_score,phy_score,coexp_score,go_score,total_score,final_score,Match
4,O00468,Q9NR23,34.0309,0.0000,34.0309,0.5125,0.0000,2.416,2.7714,5.86,684.323,684.323,True
14,A6NL08,Q9GZK3,119.9160,0.0000,119.9160,1.1877,5.2693,2.416,2.1077,0.00,3821.560,3821.560,True
47,O15230,P18054,2.4771,0.0000,2.4771,0.0000,0.0000,2.416,1.5818,105.17,995.598,995.598,True
56,O15084,Q9BZF9,299.4190,1.9704,299.4190,0.9127,0.6153,2.416,3.9008,0.00,1584.690,162488.000,True
90,A5LHX3,Q9UL46,33.1143,0.0000,33.1143,1.3730,0.6153,2.416,1.5818,5.86,626.497,626.497,True
